In [9]:
import pandas as pd

nifty50_stocks = [
    "ADANIENT.NS", "ADANIPORTS.NS", "APOLLOHOSP.NS", "ASIANPAINT.NS", "AXISBANK.NS",
    "BAJAJ-AUTO.NS", "BAJAJFINSV.NS", "BAJFINANCE.NS", "BHARTIARTL.NS", "BPCL.NS",
    "BRITANNIA.NS", "CIPLA.NS", "COALINDIA.NS", "DIVISLAB.NS", "DRREDDY.NS",
    "EICHERMOT.NS", "GRASIM.NS", "HCLTECH.NS", "HDFC.NS", "HDFCBANK.NS",
    "HDFCLIFE.NS", "HEROMOTOCO.NS", "HINDALCO.NS", "HINDUNILVR.NS", "ICICIBANK.NS",
    "INDUSINDBK.NS", "INFY.NS", "ITC.NS", "JSWSTEEL.NS", "KOTAKBANK.NS",
    "LT.NS", "M&M.NS", "MARUTI.NS", "NESTLEIND.NS", "NTPC.NS",
    "ONGC.NS", "POWERGRID.NS", "RELIANCE.NS", "SBILIFE.NS", "SHREECEM.NS",
    "SUNPHARMA.NS", "TATACONSUM.NS", "TATAMOTORS.NS", "TATASTEEL.NS", "TCS.NS",
    "TECHM.NS", "TITAN.NS", "ULTRACEMCO.NS", "UPL.NS", "WIPRO.NS"
]

banknifty_stocks = [
    "AXISBANK.NS", "BANDHANBNK.NS", "BANKBARODA.NS", "CANBK.NS", "FEDERALBNK.NS",
    "HDFCBANK.NS", "ICICIBANK.NS", "IDFCFIRSTB.NS", "INDUSINDBK.NS", "KOTAKBANK.NS",
    "PNB.NS", "SBIN.NS"
]

all_stocks = list(set(nifty50_stocks + banknifty_stocks))
print(f"Total Stocks Loaded: {len(all_stocks)}")
pd.DataFrame(all_stocks, columns=["Stock"]).to_csv("../Data/fundamental/all_stocks.csv", index=False)

Total Stocks Loaded: 57


In [10]:
import yfinance as yf
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

def fetch_fundamental_data(stock):
    try:
        ticker = yf.Ticker(stock)
        balance_sheet_q = ticker.quarterly_balance_sheet.T
        income_statement_q = ticker.quarterly_financials.T
        cashflow_q = ticker.quarterly_cashflow.T
        balance_sheet_a = ticker.balance_sheet.T
        income_statement_a = ticker.financials.T
        cashflow_a = ticker.cashflow.T
        info = ticker.info

        net_income = info.get("netIncomeToCommon")
        book_value_per_share = info.get("bookValue")
        shares_outstanding = info.get("sharesOutstanding")
        total_equity = book_value_per_share * shares_outstanding if book_value_per_share and shares_outstanding else None
        market_cap = info.get("marketCap")
        total_debt = info.get("totalDebt", 0)
        total_assets = info.get("totalAssets")

        roe = (net_income / total_equity) * 100 if net_income and total_equity else None
        roa = (net_income / total_assets) * 100 if net_income and total_assets else None
        total_investment = market_cap + total_debt if market_cap else None
        roi = (net_income / total_investment) * 100 if net_income and total_investment else None

        key_metrics = {
            "Stock": stock,
            "Market Cap": market_cap,
            "P/E Ratio": info.get("trailingPE"),
            "Forward P/E": info.get("forwardPE"),
            "PEG Ratio": info.get("trailingPegRatio"),
            "P/S Ratio": info.get("priceToSalesTrailing12Months"),
            "P/B Ratio": info.get("priceToBook"),
            "EV/EBITDA": info.get("enterpriseToEbitda"),
            "Debt to Equity": info.get("debtToEquity"),
            "ROE (%)": roe,
            "ROA (%)": roa,
            "ROI (%)": roi,
            "Gross Margin": info.get("grossMargins"),
            "Operating Margin": info.get("operatingMargins"),
            "Profit Margin": info.get("profitMargins"),
            "Dividend Yield": info.get("dividendYield"),
            "Beta": info.get("beta"),
        }

        def melt_dataframe(df, name):
            if df is not None and not df.empty:
                df = df.T
                df.reset_index(inplace=True)
                df.rename(columns={'index': 'Metric'}, inplace=True)
                df = df.melt(id_vars=['Metric'], var_name='Date', value_name='Value')
                df['Stock'] = stock
                return df
            return None

        return {
            "metrics": key_metrics,
            "balance_q": melt_dataframe(balance_sheet_q, "Balance Sheet Quarterly"),
            "income_q": melt_dataframe(income_statement_q, "Income Statement Quarterly"),
            "cashflow_q": melt_dataframe(cashflow_q, "Cash Flow Quarterly"),
            "balance_a": melt_dataframe(balance_sheet_a, "Balance Sheet Annual"),
            "income_a": melt_dataframe(income_statement_a, "Income Statement Annual"),
            "cashflow_a": melt_dataframe(cashflow_a, "Cash Flow Annual"),
        }
    except Exception as e:
        print(f"Failed to fetch data for {stock}: {e}")
        return None


all_data = []
with ThreadPoolExecutor(max_workers=5) as executor:
    all_data = list(executor.map(fetch_fundamental_data, all_stocks))

all_data = [d for d in all_data if d]

df_metrics = pd.DataFrame([d["metrics"] for d in all_data])
df_balance_q = pd.concat([d["balance_q"] for d in all_data if d["balance_q"] is not None], ignore_index=True)
df_income_q = pd.concat([d["income_q"] for d in all_data if d["income_q"] is not None], ignore_index=True)
df_cashflow_q = pd.concat([d["cashflow_q"] for d in all_data if d["cashflow_q"] is not None], ignore_index=True)
df_balance_a = pd.concat([d["balance_a"] for d in all_data if d["balance_a"] is not None], ignore_index=True)
df_income_a = pd.concat([d["income_a"] for d in all_data if d["income_a"] is not None], ignore_index=True)
df_cashflow_a = pd.concat([d["cashflow_a"] for d in all_data if d["cashflow_a"] is not None], ignore_index=True)

df_metrics.to_csv("../Data/fundamental/fundamental_metrics.csv", index=False)
df_balance_q.to_csv("../Data/fundamental/quarterly_balance_sheet.csv", index=False)
df_income_q.to_csv("../Data/fundamental/quarterly_income_statement.csv", index=False)
df_cashflow_q.to_csv("../Data/fundamental/quarterly_cashflow.csv", index=False)
df_balance_a.to_csv("../Data/fundamental/annual_balance_sheet.csv", index=False)
df_income_a.to_csv("../Data/fundamental/annual_income_statement.csv", index=False)
df_cashflow_a.to_csv("../Data/fundamental/annual_cashflow.csv", index=False)

In [11]:
# import pandas as pd
# from google.colab import files

# def get_date_range(file_name):
#     df = pd.read_csv(file_name)
#     if 'Date' in df.columns:
#         df['Date'] = pd.to_datetime(df['Date'], errors='coerce').dt.date
#         min_date = df['Date'].min()
#         max_date = df['Date'].max()

#         if 'annual' in file_name.lower():
#             years = max_date.year - min_date.year
#             if max_date.month > min_date.month:
#                 years += 1
#             print(f"{file_name}: {min_date} to {max_date}")
#             print(f"Years: {years}")
#         elif 'quarterly' in file_name.lower():
#             quarters = (max_date.year - min_date.year) * 4 + (max_date.month - 1) // 3 + 1
#             print(f"{file_name}: {min_date} to {max_date}")
#             print(f"Quarters: {quarters}")
#     else:
#         print(f"No 'Date' column found in {file_name}")

# get_date_range("../Data/fundamental/annual_income_statement.csv")
# get_date_range("../Data/fundamental/annual_balance_sheet.csv")
# get_date_range("../Data/fundamental/annual_cashflow.csv")
# get_date_range("../Data/fundamental/quarterly_income_statement.csv")
# get_date_range("../Data/fundamental/quarterly_balance_sheet.csv")
# get_date_range("../Data/fundamental/quarterly_cashflow.csv")


In [12]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import time

# Selecting required features
selected_features = ["Metric", "Date", "Value", "Stock"]
df_income_q = df_income_q[selected_features]
df_balance_q = df_balance_q[selected_features]
df_cashflow_q = df_cashflow_q[selected_features]

# Pivot dataframes
df_income_pivot = df_income_q.pivot_table(index=["Date", "Stock"], columns="Metric", values="Value").reset_index()
df_balance_pivot = df_balance_q.pivot_table(index=["Date", "Stock"], columns="Metric", values="Value").reset_index()
df_cashflow_pivot = df_cashflow_q.pivot_table(index=["Date", "Stock"], columns="Metric", values="Value").reset_index()

# Ensure only relevant columns from df_metrics are merged
df_metrics_selected = df_metrics[["Stock", "Market Cap", "P/E Ratio", "Forward P/E", "PEG Ratio",
                                  "P/S Ratio", "P/B Ratio", "EV/EBITDA", "Debt to Equity",
                                  "ROE (%)", "ROA (%)", "ROI (%)", "Gross Margin",
                                  "Operating Margin", "Profit Margin", "Dividend Yield", "Beta"]]

# Merge the fundamental metrics with the financial statements
df_merged = df_income_pivot.merge(df_balance_pivot, on=["Date", "Stock"], how="outer", suffixes=("", "_bal"))
df_merged = df_merged.merge(df_cashflow_pivot, on=["Date", "Stock"], how="outer", suffixes=("", "_cf"))

# Merge stock-level metrics (df_metrics) with df_merged on "Stock"
df_merged = df_merged.merge(df_metrics_selected, on="Stock", how="left")

# Sort and fill missing values
df_merged = df_merged.sort_values(by=["Stock", "Date"]).infer_objects(copy=False).fillna(0)

# Scale all numerical columns except "Stock" and "Date"
scaler = MinMaxScaler()
df_scaled = df_merged.copy()
df_scaled.iloc[:, 2:] = scaler.fit_transform(df_scaled.iloc[:, 2:])



/var/folders/kq/6mphs8bd5d19hyzf97x081k00000gp/T/ipykernel_56511/3480108562.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_merged = df_merged.sort_values(by=["Stock", "Date"]).infer_objects(copy=False).fillna(0)


# LSTM Quarterly HyperParameter Tuning (Dont run this)

In [13]:
import keras_tuner as kt

def create_sequences(data, stock_col, target_col, seq_length=4):
    sequences, labels, stocks = [], [], []
    for stock in data[stock_col].unique():
        stock_data = data[data[stock_col] == stock].drop(columns=[stock_col, "Date"]).values
        if len(stock_data) > seq_length:
            for i in range(len(stock_data) - seq_length):
                sequences.append(stock_data[i:i+seq_length])
                labels.append(stock_data[i+seq_length, target_col])
                stocks.append(stock)
    return np.array(sequences), np.array(labels), stocks

X, y, stock_list = create_sequences(df_scaled, "Stock", target_col=2)

X_train, X_test, y_train, y_test, stock_train, stock_test = train_test_split(
    X, y, stock_list, test_size=0.3, random_state=42, shuffle=True
)

def build_model(hp):
    model = Sequential()
    model.add(tf.keras.Input(shape=(X_train.shape[1], X_train.shape[2])))

    # First BiLSTM layer
    model.add(Bidirectional(LSTM(hp.Int('bilstm_units_1', min_value=32, max_value=128, step=32), return_sequences=True)))
    model.add(Dropout(hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)))

    # Second BiLSTM layer
    model.add(Bidirectional(LSTM(hp.Int('bilstm_units_2', min_value=32, max_value=128, step=32))))
    model.add(Dropout(hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)))

    # Output layer
    model.add(Dense(1))

    # Compile model
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                  loss='mse')

    return model


tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    directory='../Data/fundamental/hyperparameter_tuning',
    project_name='lstm_stock_prediction'
)

tuner.search(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test), verbose=1)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)

start_train = time.time()
best_model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test))
end_train = time.time()
training_time = end_train - start_train

start_pred = time.time()
predictions = best_model.predict(X_test)
end_pred = time.time()
prediction_time = end_pred - start_pred

def rank_top_stocks(preds, stocks, top_n=10):
    stock_preds = pd.DataFrame({"Stock": stocks, "Predicted_Net_Income": preds.flatten()})
    stock_preds["Company"] = stock_preds["Stock"].str.replace(r'\.NS|\.BO', '', regex=True)
    stock_avg_preds = stock_preds.groupby("Company", as_index=False)["Predicted_Net_Income"].mean()
    ranked_stocks = stock_avg_preds.sort_values(by="Predicted_Net_Income", ascending=False).head(top_n).reset_index(drop=True)
    ranked_stocks.index += 1
    ranked_stocks.reset_index(inplace=True)
    ranked_stocks.rename(columns={"index": "S.No"}, inplace=True)
    return ranked_stocks

top_stocks = rank_top_stocks(predictions, stock_test)
print(top_stocks)
print(f"Total Training Time: {training_time:.2f} seconds")
print(f"Total Prediction Time: {prediction_time:.2f} seconds")

Trial 10 Complete [00h 00m 07s]
val_loss: 0.022716928273439407

Best val_loss So Far: 0.011405902914702892
Total elapsed time: 00h 01m 18s
Epoch 1/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - loss: 1.2622 - val_loss: 0.1049
Epoch 2/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.1042 - val_loss: 0.0706
Epoch 3/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0860 - val_loss: 0.0636
Epoch 4/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0847 - val_loss: 0.0623
Epoch 5/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0807 - val_loss: 0.0599
Epoch 6/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0704 - val_loss: 0.0581
Epoch 7/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0713 - val_loss: 0.0549
Epoch 8/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0633 - val_loss: 0.0466
Epoch 9/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0517 - val_loss: 0.0458
Epoch 10/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0408 - val_loss: 0.0747
Epoch 11/100
8/8 ━━━━━━━━━━━━

# Bi-LSTM Quarterly model with best parameters

In [14]:

# Function to create sequences for LSTM input
def create_sequences(data, stock_col, target_col, seq_length=4):
    sequences, labels, stocks = [], [], []
    for stock in data[stock_col].unique():
        stock_data = data[data[stock_col] == stock].drop(columns=[stock_col, "Date"]).values
        if len(stock_data) > seq_length:
            for i in range(len(stock_data) - seq_length):
                sequences.append(stock_data[i:i+seq_length])
                labels.append(stock_data[i+seq_length, target_col])
                stocks.append(stock)
    return np.array(sequences), np.array(labels), stocks

X, y, stock_list = create_sequences(df_scaled, "Stock", target_col=285)

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test, stock_train, stock_test = train_test_split(
    X, y, stock_list, test_size=0.3, random_state=42, shuffle=True
)

# Define BiLSTM model
model = Sequential([
    tf.keras.Input(shape=(X_train.shape[1], X_train.shape[2])),
    Bidirectional(LSTM(96, return_sequences=True)),  # BiLSTM layer 1
    Dropout(0.3),
    Bidirectional(LSTM(96)),  # BiLSTM layer 2
    Dropout(0.2),
    Dense(1)
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.004201765072917902), loss='mse')

# Train the model
start_train = time.time()
model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test))
end_train = time.time()
training_time = end_train - start_train

# Make predictions
start_pred = time.time()
predictions = model.predict(X_test)
end_pred = time.time()
prediction_time = end_pred - start_pred

# Rank top stocks based on predictions
def rank_top_stocks(preds, stocks, top_n=10):
    stock_preds = pd.DataFrame({"Stock": stocks, "Predicted_Net_Income": preds.flatten()})
    stock_preds["Company"] = stock_preds["Stock"].str.replace(r'\.NS|\.BO', '', regex=True)
    stock_avg_preds = stock_preds.groupby("Company", as_index=False)["Predicted_Net_Income"].mean()
    ranked_stocks = stock_avg_preds.sort_values(by="Predicted_Net_Income", ascending=False).head(top_n).reset_index(drop=True)
    ranked_stocks.index += 1
    ranked_stocks.reset_index(inplace=True)
    ranked_stocks.rename(columns={"index": "S.No"}, inplace=True)
    return ranked_stocks[["S.No", "Company", "Predicted_Net_Income"]]

# Get top ranked stocks
top_stocks = rank_top_stocks(predictions, stock_test)
print(top_stocks)

# Print training and prediction times
print(f"Total Training Time: {training_time:.2f} seconds")
print(f"Total Prediction Time: {prediction_time:.2f} seconds")


Epoch 1/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - loss: 0.7207 - val_loss: 0.1653
Epoch 2/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0711 - val_loss: 0.0223
Epoch 3/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0533 - val_loss: 0.0179
Epoch 4/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0307 - val_loss: 0.0148
Epoch 5/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0295 - val_loss: 0.0199
Epoch 6/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0284 - val_loss: 0.0145
Epoch 7/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0267 - val_loss: 0.0121
Epoch 8/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0255 - val_loss: 0.0117
Epoch 9/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0251 - val_loss: 0.0122
Epoch 10/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0237 - val_loss: 0.0112
Epoch 11/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0217 - val_loss: 0.0109
Epoch 12/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0212 - val_loss: 

In [ ]:
# Making predictions
start_pred = time.time()
predictions = model.predict(X_test)
end_pred = time.time()
prediction_time = end_pred - start_pred

def rank_top_stocks(preds, stocks, top_n=10):
    stock_preds = pd.DataFrame({"Stock": stocks, "Predicted_Net_Income": preds.flatten()})
    stock_preds["Company"] = stock_preds["Stock"].str.replace(r'\.NS|\.BO', '', regex=True)
    stock_avg_preds = stock_preds.groupby("Company", as_index=False)["Predicted_Net_Income"].mean()
    ranked_stocks = stock_avg_preds.sort_values(by="Predicted_Net_Income", ascending=False).head(top_n).reset_index(drop=True)
    ranked_stocks.index += 1
    ranked_stocks.reset_index(inplace=True)
    ranked_stocks.rename(columns={"index": "S.No"}, inplace=True)
    return ranked_stocks[["S.No", "Company", "Predicted_Net_Income"]]

top_stocks = rank_top_stocks(predictions, stock_test)
top_stocks.to_csv("../Data/fundamental/top_stocks.csv", index=False)
top_stocks

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


,S.No,Company,Predicted_Net_Income
0,1,BRITANNIA,0.586954
1,2,COALINDIA,0.429983
2,3,BHARTIARTL,0.371957
3,4,ITC,0.346446
4,5,BAJAJ-AUTO,0.286892
5,6,EICHERMOT,0.280245
6,7,HINDUNILVR,0.273297
7,8,BAJFINANCE,0.239618
8,9,CIPLA,0.231883
9,10,ASIANPAINT,0.227434


In [16]:
print(f"Total Training Time: {training_time:.2f} seconds")
print(f"Total Prediction Time: {prediction_time:.2f} seconds")

Total Training Time: 13.06 seconds
Total Prediction Time: 0.03 seconds


In [17]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error, explained_variance_score

mae = mean_absolute_error(y_test, predictions)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
r2 = r2_score(y_test, predictions)
mape = mean_absolute_percentage_error(y_test, predictions)
explained_var = explained_variance_score(y_test, predictions)

print(f"Model Evaluation:")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Percentage Error (MAPE): {mape * 100:.2f}%")
print(f"R² Score: {r2}")
print(f"Explained Variance Score: {explained_var}")

def interpret_r2_score(r2):
    if r2 >= 0.9:
        return "Excellent Model Fit"
    elif r2 >= 0.75:
        return "Good Model Fit"
    elif r2 >= 0.5:
        return "Moderate Model Fit"
    else:
        return "Poor Model Fit - Consider Improving"

print(f"\nModel Fit Interpretation: {interpret_r2_score(r2)}")

Model Evaluation:
Mean Absolute Error (MAE): 0.02384786597759315
Root Mean Squared Error (RMSE): 0.028677821135479564
Mean Absolute Percentage Error (MAPE): 27.19%
R² Score: 0.9343213767716119
Explained Variance Score: 0.9500109729499081

Model Fit Interpretation: Excellent Model Fit
